# EE 297 - HW1: CIFAR-10

This notebook was completed as a course requirement for EE297: Deep Learning in the University of the Philippines Diliman.

## Objectives:
1. Train a 3-layer MLP on the CIFAR-10 dataset to acheive the highest accuracy possible. Tune the hyperparameters, implement regularization and apply data augmentation to improve your results
2. Train a 3-layer Convolutional Neural Network on the CIFAR-10 dataset to acheive the highest accuracy possible. Tune the hyperparameters, implement regularization and apply data augmentation to improve your results

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
import numpy as np

import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim

from tensorboardX import SummaryWriter # Pytorch + Tensorboard integration

# I. Data Loading and Augmentation

## Cutout Regularizer

Cutout regularization randomly masks patches from the image before training. This improves regularization since the networks learns to classify even with sections blacked out. This regularizer also results to train loss typically being higher than test loss since the training domain is harder due to blacked out patches

In [2]:
# https://github.com/uoguelph-mlrg/Cutout/blob/master/util/cutout.py
class Cutout(object):
    """Randomly mask out one or more patches from an image.
    Args:
        n_holes (int): Number of patches to cut out of each image.
        length (int): The length (in pixels) of each square patch.
    """
    def __init__(self, n_holes, length):
        self.n_holes = n_holes
        self.length = length

    def __call__(self, img):
        """
        Args:
            img (Tensor): Tensor image of size (C, H, W).
        Returns:
            Tensor: Image with n_holes of dimension length x length cut out of it.
        """
        h = img.size(1)
        w = img.size(2)

        mask = np.ones((h, w), np.float32)

        for n in range(self.n_holes):
            y = np.random.randint(h)
            x = np.random.randint(w)

            y1 = np.clip(y - self.length // 2, 0, h)
            y2 = np.clip(y + self.length // 2, 0, h)
            x1 = np.clip(x - self.length // 2, 0, w)
            x2 = np.clip(x + self.length // 2, 0, w)

            mask[y1: y2, x1: x2] = 0.

        mask = torch.from_numpy(mask)
        mask = mask.expand_as(img)
        img = img * mask

        return img

# Load Dataset and Apply Data Augmentation

In [2]:
normalize = transforms.Normalize(mean=[x / 255.0 for x in [125.3, 123.0, 113.9]], std=[x / 255.0 for x in [63.0, 62.1, 66.7]])

train_transform = transforms.Compose([])

train_transform.transforms.append(transforms.RandomCrop(32, padding=4))
train_transform.transforms.append(transforms.RandomHorizontalFlip())
train_transform.transforms.append(transforms.ToTensor())
train_transform.transforms.append(normalize)
#train_transform.transforms.append(Cutout(n_holes=1, length=16))

test_transform = transforms.Compose([
    transforms.ToTensor(),
    normalize])

trainset = torchvision.datasets.CIFAR10(root='/home/josen/datasets/CIFAR/data', train=True,
                                        download=True, transform=train_transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='/home/josen/datasets/CIFAR/data', train=False,
                                       download=True, transform=test_transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
# functions to show an image
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join('%5s' % classes[labels[j]] for j in range(4)))

# II. Network Definitions

## 3-Layer Multilayer Perceptron

In [6]:
class FCNet(nn.Module):
    def __init__(self):
        super(FCNet, self).__init__()
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(32*32*3, 1024)
        self.fc2 = nn.Linear(1024, 256)
        self.fc3 = nn.Linear(256, 10)

    def forward(self, x):
        
        x = x.view(-1, 32*32*3) # Flatten
        x = F.relu(self.fc1(x)) # Layer 1
        x = F.dropout(x)
        
        x = F.relu(self.fc2(x)) # Layer 2
        x = F.dropout(x)
        
        x = self.fc3(x) # Layer 3
        
        return x


fcnet = FCNet()

## 3-Layer Convolutional Neural Network

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3,64, 3)
        self.bn1 = nn.BatchNorm2d(64)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(64, 128, 3)
        self.bn2 = nn.BatchNorm2d(128)
        self.conv3 = nn.Conv2d(128, 512, 3)
        self.bn3 = nn.BatchNorm2d(512)
        self.fc1 = nn.Linear(512*6*6, 10)

    def forward(self, x):
        x = F.dropout(F.relu(self.bn1(self.conv1(x))), p = 0.2)
        x = self.pool(F.dropout(F.relu(self.bn2(self.conv2(x))), p = 0.2))
        x = self.pool(F.dropout(F.relu(self.bn3(self.conv3(x))), p = 0.2))
        x = x.view(-1, 512*6*6)
        x = self.fc1(x)
        return x


net = Net()

# III. Training

## 3-Layer Multilayer Perceptron Training

In [5]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(fcnet.parameters(), lr=0.001, nesterov=True, weight_decay=5e-4, momentum = 0.9)
writer = SummaryWriter()

total_batches = 0
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[30, 60, 80], gamma=0.1)

for epoch in range(90):  # loop over the dataset multiple times
    
    print('Epoch ' + str(epoch))
    scheduler.step(epoch) # Adjust learning rate
    
    running_loss = 0.0
    
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
                
        # CUDA    
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        fcnet.to(device)
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = fcnet(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 400 == 399:    # print every 400 mini-batches
            print('Train Loss: %.3f' %
                  (running_loss / 400))
            writer.add_scalar('data/fcnet__train_loss', running_loss / 400, total_batches)
            running_loss = 0.0
            total_batches += 400
            
    total = 0
    correct = 0
    with torch.no_grad():
        running_test_loss = 0.0
        for data_test in testloader:
            images_test, labels_test = data_test
            images_test, labels_test = images_test.to(device), labels_test.to(device)
            outputs_test = fcnet(images_test)
            _, predicted_test = torch.max(outputs_test.data, 1)
            total += labels_test.size(0)
            correct += (predicted_test == labels_test).sum().item()
            
            running_test_loss += criterion(outputs_test, labels_test)
            
        writer.add_scalar('data/fcnet_test_acc', 100 * correct / total, epoch)
        writer.add_scalar('data/fcnet_test_loss', running_test_loss / len(testloader), epoch)
        
        print("Test Loss: %.3f" % (running_test_loss/len(testloader)))        
        print("Test Accuracy: %.3f" % (100 * correct / total))


print('Finished Training')

Epoch 0
Train Loss: 2.172
Train Loss: 2.043
Train Loss: 2.005
Test Loss: 1.870
Test Accuracy: 33.080
Epoch 1
Train Loss: 1.927
Train Loss: 1.911
Train Loss: 1.916
Test Loss: 1.793
Test Accuracy: 36.060
Epoch 2
Train Loss: 1.870
Train Loss: 1.858
Train Loss: 1.868
Test Loss: 1.774
Test Accuracy: 36.820
Epoch 3
Train Loss: 1.842
Train Loss: 1.847
Train Loss: 1.836
Test Loss: 1.733
Test Accuracy: 38.690
Epoch 4
Train Loss: 1.802
Train Loss: 1.816
Train Loss: 1.808
Test Loss: 1.707
Test Accuracy: 39.960
Epoch 5
Train Loss: 1.787
Train Loss: 1.781
Train Loss: 1.794
Test Loss: 1.690
Test Accuracy: 40.080
Epoch 6
Train Loss: 1.771
Train Loss: 1.792
Train Loss: 1.767
Test Loss: 1.678
Test Accuracy: 40.330
Epoch 7
Train Loss: 1.763
Train Loss: 1.760
Train Loss: 1.758
Test Loss: 1.646
Test Accuracy: 41.900
Epoch 8
Train Loss: 1.744
Train Loss: 1.760
Train Loss: 1.743
Test Loss: 1.658
Test Accuracy: 41.470
Epoch 9
Train Loss: 1.724
Train Loss: 1.741
Train Loss: 1.752
Test Loss: 1.636
Test Accurac

Train Loss: 1.552
Test Loss: 1.507
Test Accuracy: 46.100
Epoch 81
Train Loss: 1.545
Train Loss: 1.574
Train Loss: 1.547
Test Loss: 1.504
Test Accuracy: 46.350
Epoch 82
Train Loss: 1.552
Train Loss: 1.542
Train Loss: 1.560
Test Loss: 1.517
Test Accuracy: 46.060
Epoch 83
Train Loss: 1.550
Train Loss: 1.553
Train Loss: 1.565
Test Loss: 1.511
Test Accuracy: 46.200
Epoch 84
Train Loss: 1.549
Train Loss: 1.567
Train Loss: 1.550
Test Loss: 1.505
Test Accuracy: 46.380
Epoch 85
Train Loss: 1.554
Train Loss: 1.556
Train Loss: 1.569
Test Loss: 1.510
Test Accuracy: 46.410
Epoch 86
Train Loss: 1.555
Train Loss: 1.546
Train Loss: 1.557
Test Loss: 1.504
Test Accuracy: 46.980
Epoch 87
Train Loss: 1.566
Train Loss: 1.558
Train Loss: 1.552
Test Loss: 1.502
Test Accuracy: 47.040
Epoch 88
Train Loss: 1.553
Train Loss: 1.545
Train Loss: 1.552
Test Loss: 1.510
Test Accuracy: 46.640
Epoch 89
Train Loss: 1.545
Train Loss: 1.570
Train Loss: 1.547
Test Loss: 1.513
Test Accuracy: 46.000
Finished Training


## 3-Layer Convolutional Neural Network Training

In [4]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, nesterov=True, weight_decay=5e-4, momentum = 0.9)
writer = SummaryWriter()

total_batches = 0
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[50, 100, 150], gamma=0.1)

for epoch in range(200):  # loop over the dataset multiple times
    
    print('Epoch ' + str(epoch))
    scheduler.step(epoch) # Adjust learning rate
    
    running_loss = 0.0
    
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
                
        # CUDA    
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        net.to(device)
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 400 == 399:    # print every 400 mini-batches
            print('Train Loss: %.3f' %
                  (running_loss / 400))
            writer.add_scalar('data/net__train_loss', running_loss / 400, total_batches)
            running_loss = 0.0
            total_batches += 400
            
    total = 0
    correct = 0
    with torch.no_grad():
        running_test_loss = 0.0
        for data_test in testloader:
            images_test, labels_test = data_test
            images_test, labels_test = images_test.to(device), labels_test.to(device)
            outputs_test = net(images_test)
            _, predicted_test = torch.max(outputs_test.data, 1)
            total += labels_test.size(0)
            correct += (predicted_test == labels_test).sum().item()
            
            running_test_loss += criterion(outputs_test, labels_test)
            
        writer.add_scalar('data/net_test_acc', 100 * correct / total, epoch)
        writer.add_scalar('data/net_test_loss', running_test_loss / len(testloader), epoch)
        
        print("Test Loss: %.3f" % (running_test_loss/len(testloader)))        
        print("Test Accuracy: %.3f" % (100 * correct / total))


print('Finished Training')

Epoch 0
Train Loss: 1.802
Train Loss: 1.554
Train Loss: 1.416
Test Loss: 1.192
Test Accuracy: 58.090
Epoch 1
Train Loss: 1.269
Train Loss: 1.226
Train Loss: 1.177
Test Loss: 1.015
Test Accuracy: 64.560
Epoch 2
Train Loss: 1.146
Train Loss: 1.094
Train Loss: 1.100
Test Loss: 0.898
Test Accuracy: 68.850
Epoch 3
Train Loss: 1.033
Train Loss: 1.031
Train Loss: 1.015
Test Loss: 0.945
Test Accuracy: 67.280
Epoch 4
Train Loss: 0.973
Train Loss: 0.972
Train Loss: 0.953
Test Loss: 0.815
Test Accuracy: 71.980
Epoch 5
Train Loss: 0.922
Train Loss: 0.935
Train Loss: 0.903
Test Loss: 0.834
Test Accuracy: 71.260
Epoch 6
Train Loss: 0.888
Train Loss: 0.889
Train Loss: 0.878
Test Loss: 0.832
Test Accuracy: 71.030
Epoch 7
Train Loss: 0.840
Train Loss: 0.858
Train Loss: 0.862
Test Loss: 0.739
Test Accuracy: 74.420
Epoch 8
Train Loss: 0.812
Train Loss: 0.823
Train Loss: 0.800
Test Loss: 0.737
Test Accuracy: 74.530
Epoch 9
Train Loss: 0.783
Train Loss: 0.779
Train Loss: 0.780
Test Loss: 0.698
Test Accurac

Train Loss: 0.352
Test Loss: 0.445
Test Accuracy: 84.780
Epoch 81
Train Loss: 0.354
Train Loss: 0.356
Train Loss: 0.351
Test Loss: 0.446
Test Accuracy: 85.240
Epoch 82
Train Loss: 0.347
Train Loss: 0.359
Train Loss: 0.356
Test Loss: 0.452
Test Accuracy: 84.620
Epoch 83
Train Loss: 0.344
Train Loss: 0.347
Train Loss: 0.358
Test Loss: 0.446
Test Accuracy: 84.890
Epoch 84
Train Loss: 0.354
Train Loss: 0.347
Train Loss: 0.356
Test Loss: 0.447
Test Accuracy: 84.930
Epoch 85
Train Loss: 0.353
Train Loss: 0.345
Train Loss: 0.350
Test Loss: 0.450
Test Accuracy: 84.730
Epoch 86
Train Loss: 0.349
Train Loss: 0.351
Train Loss: 0.352
Test Loss: 0.447
Test Accuracy: 84.920
Epoch 87
Train Loss: 0.350
Train Loss: 0.348
Train Loss: 0.346
Test Loss: 0.453
Test Accuracy: 84.600
Epoch 88
Train Loss: 0.345
Train Loss: 0.344
Train Loss: 0.341
Test Loss: 0.450
Test Accuracy: 84.590
Epoch 89
Train Loss: 0.353
Train Loss: 0.338
Train Loss: 0.349
Test Loss: 0.448
Test Accuracy: 84.740
Epoch 90
Train Loss: 0.35

Train Loss: 0.330
Train Loss: 0.332
Test Loss: 0.442
Test Accuracy: 85.060
Epoch 161
Train Loss: 0.326
Train Loss: 0.322
Train Loss: 0.326
Test Loss: 0.438
Test Accuracy: 85.190
Epoch 162
Train Loss: 0.326
Train Loss: 0.328
Train Loss: 0.332
Test Loss: 0.445
Test Accuracy: 84.820
Epoch 163
Train Loss: 0.329
Train Loss: 0.342
Train Loss: 0.325
Test Loss: 0.442
Test Accuracy: 85.280
Epoch 164
Train Loss: 0.327
Train Loss: 0.320
Train Loss: 0.323
Test Loss: 0.445
Test Accuracy: 84.970
Epoch 165
Train Loss: 0.337
Train Loss: 0.316
Train Loss: 0.325
Test Loss: 0.444
Test Accuracy: 84.960
Epoch 166
Train Loss: 0.324
Train Loss: 0.327
Train Loss: 0.332
Test Loss: 0.442
Test Accuracy: 85.290
Epoch 167
Train Loss: 0.320
Train Loss: 0.329
Train Loss: 0.327
Test Loss: 0.439
Test Accuracy: 85.190
Epoch 168
Train Loss: 0.334
Train Loss: 0.324
Train Loss: 0.324
Test Loss: 0.442
Test Accuracy: 84.940
Epoch 169
Train Loss: 0.328
Train Loss: 0.326
Train Loss: 0.327
Test Loss: 0.440
Test Accuracy: 85.11

In [ ]:
dataiter = iter(testloader)
images, labels = dataiter.next()
images_cuda, labels_cuda = images.to(device), labels.to(device)

# print images
imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(4)))

outputs = net(images_cuda)

# Final Results

## CIFAR-10 on 3-Layer MLP
1. No data augmentation = 52.2%
2. With data augmentation and Cutout = 46.64%

Cutout probably makes the dataset too hard for MLP. Increasing the capacity of MLP may be able to improve accuracy

## CIFAR-10 on 3-Layer CNN
1. No data augmentation = 73.57%
2. With Data Augmentation and Cutout = 84.42% (Epoch 151)
3. With Data Augmentation, No Cutout, More Epochs, More Filters = 85.55% (Epoch 151)